# Managing an Azure Kubernetes Service cluster from Cloud Shell

So we set up our AKS cluser, let's go ahead and run the import setup code from the last notbook and then do some troubleshooting work.

In [ ]:
Enter-AzShell

In [ ]:
Set-AzContext -SubscriptionName 'Oaas-SubLib-013' | Out-Null
Import-AzAksCredential -ResourceGroupName manning-test-aks -Name manning-test-aks -Force

## Kubernetes Troubleshooting

> NOTE: This text was shamelessly stolen from [this set of docs by a Microsoft engineer](https://github.com/feiskyer/kubernetes-handbook/blob/master/en/troubleshooting/index.md).

This part introduces how to troubleshoot various problems on Kubernetes, includes

- [Troubleshooting Clusters](cluster.md)
- [Troubleshooting Pods](pod.md)
- [Troubleshooting Networking](network.md)
- [Troubleshooting Persistent Volumes](pv.md)
  - [AzureDisk](azuredisk.md)
  - [AzureFile](azurefile.md)
- [Troubleshooting Windows Containers](windows.md)
- [Troubleshooting Cloud Provider](cloud.md)
  - [Azure](azure.md)
- [Tools for Troubleshooting](tools.md)

Remember `kubectl` is always the most important tool when starting to troubleshoot any problems.

### Listing Nodes

In [ ]:
kubectl get nodes

$nodename = kubectl get nodes -o=jsonpath='{.items[0].metadata.name}'
kubectl describe node $nodename

### Listing Pods

In [ ]:
kubectl get pods -o wide
kubectl -n kube-system get pods -o wide

### Looking at Pod events

In [ ]:
$podname = kubectl get pods -o=jsonpath='{.items[0].metadata.name}'
kubectl describe pod $podname

### Looking at kube-apiserver logs

In [ ]:
# Looking at kube-apiserver logs
$apiserverPodname = kubectl -n kube-system get pod -l component=kube-apiserver -o jsonpath='{.items[0].metadata.name}'
if ($apiserverPodname) {
    kubectl -n kube-system logs $apiserverPodname --tail 100
}

# Looking at kube-controller-manager logs
$controllerPodname = kubectl -n kube-system get pod -l component=kube-controller-manager -o jsonpath='{.items[0].metadata.name}'
if ($controllerPodname) {
    kubectl -n kube-system logs $controllerPodname --tail 100
}

# Looking at kube-scheduler logs
$schedulerPodname = kubectl -n kube-system get pod -l component=kube-scheduler -o jsonpath='{.items[0].metadata.name}'
if ($schedulerPodname) {
    kubectl -n kube-system logs $schedulerPodname --tail 100
}

# Looking at kube-dns logs
$dnsPodname = kubectl -n kube-system get pod -l k8s-app=kube-dns -o jsonpath='{.items[0].metadata.name}'
if ($dnsPodname) {
    kubectl -n kube-system logs $dnsPodname -c kubedns
}


### Looking at kube-proxy logs

In [ ]:
kubectl -n kube-system get pod -l component=kube-proxy

$podname = kubectl -n kube-system get pod -l component=kube-proxy -o=jsonpath='{.items[0].metadata.name}'
kubectl -n kube-system logs $podname